In [ ]:
!pip install pandas "mistralai>=0.1.2"

In [ ]:
import pandas as pd

# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "retrieve_payment_status",
      "description": "Get payment status of a transaction",
      "parameters": {
        "type": "object",
        "properties": {
          "transaction_id": {
            "type": "string",
            "description": "The transaction id."
          }
        },
        "required": ["transaction_id"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "retrieve_payment_date",
      "description": "Get payment date of a transaction",
      "parameters": {
        "type": "object",
        "properties": {
          "transaction_id": {
            "type": "string",
            "description": "The transaction id."
          }
        },
        "required": ["transaction_id"]
      }
    }
  }
]


In [ ]:
import functools

names_to_functions = {
    'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
    'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}


In [ ]:
from mistralai.models.chat_completion import ChatMessage

messages = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]


In [ ]:
from mistralai.client import MistralClient

model = "open-mixtral-8x22b"
api_key="xxx"

client = MistralClient(api_key=api_key)

response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response

In [ ]:
messages.append(ChatMessage(role="assistant", content=response.choices[0].message.content))
messages.append(ChatMessage(role="user", content="My transaction ID is T1001."))

In [ ]:
messages

In [ ]:
response = client.chat(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

response

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
messages

In [ ]:
import json
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)

In [ ]:
function_result = names_to_functions[function_name](**function_params)
function_result


In [ ]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result))

In [ ]:
messages

In [ ]:
response = client.chat(
    model=model,
    messages=messages
)

response.choices[0].message.content